In [1]:
import torch
import torch.multiprocessing as mp
mp.set_start_method("spawn")
from torch.utils.data import TensorDataset, DataLoader
import deepadr
from deepadr.hyphelper import *

In [2]:
# inps = torch.randn(20, 15, 100, dtype=torch.float32)
# tgts = torch.randn(20, 15, 1, dtype=torch.float32)
# dataset = TensorDataset(inps, tgts)

# loader = DataLoader(dataset, batch_size=2)

from torch_geometric.datasets import *
from torch_geometric.loader import DataLoader

dataset = TUDataset(root="../data/raw/PyG/TUDataset/", name="MUTAG")
loader = DataLoader(dataset, batch_size=32, shuffle=True)


    
def spawn_q_process(q_process):
    print(">>> spawning process")
    q_process.start()
    
def join_q_process(q_process):
    q_process.join()
    print("<<< joined process")
    

def create_q_process(loader, queue):
    return mp.Process(target=deepadr.hyphelper.test_mp, args=(loader, queue))



queue = mp.Queue()
q_processes = []
num_partitions = 1

for q_i in range(num_partitions):
    print('run_num:', q_i)
    q_process = create_q_process(loader=loader, 
                                 queue=queue)
    q_processes.append(q_process)
    spawn_q_process(q_process)
    
print("*"*25)
for q_i in range(num_partitions):
    join_q_process(q_processes[q_i])
    released_gpu_num = queue.get()
    

run_num: 0
>>> spawning process
*************************
len(loader): 6
len(loader.dataset): 188
---------
<<< joined process
